# Financial Intelligence System Analysis

This notebook demonstrates how to analyze the transaction data and risk patterns using our system's APIs.

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
from datetime import datetime, timedelta

# Configure API endpoints
BASE_URL = 'http://localhost:5001'
ANALYSIS_URL = 'http://localhost:5002'
SCORING_URL = 'http://localhost:5003'

## 1. Load and Analyze Transaction Data

In [ ]:
def get_transactions(start_date=None, end_date=None, min_amount=None):
    params = {}
    if start_date:
        params['start_date'] = start_date
    if end_date:
        params['end_date'] = end_date
    if min_amount:
        params['min_amount'] = min_amount
        
    response = requests.get(f'{BASE_URL}/api/v1/transactions/search', params=params)
    return pd.DataFrame(response.json())

# Get last 24 hours of transactions
end_date = datetime.now().isoformat()
start_date = (datetime.now() - timedelta(days=1)).isoformat()
df_transactions = get_transactions(start_date, end_date)

print(f'Loaded {len(df_transactions)} transactions')
df_transactions.head()

## 2. Analyze Transaction Patterns

In [ ]:
# Transaction amount distribution
plt.figure(figsize=(12, 6))
sns.histplot(data=df_transactions, x='amount', bins=50)
plt.title('Transaction Amount Distribution')
plt.xlabel('Amount')
plt.ylabel('Count')
plt.show()

# Transaction volume over time
df_transactions['timestamp'] = pd.to_datetime(df_transactions['timestamp'])
hourly_volume = df_transactions.resample('H', on='timestamp')['amount'].sum()

plt.figure(figsize=(12, 6))
hourly_volume.plot()
plt.title('Transaction Volume by Hour')
plt.xlabel('Hour')
plt.ylabel('Total Amount')
plt.show()

## 3. Analyze Risk Patterns

In [ ]:
def get_risk_metrics():
    response = requests.get(f'{SCORING_URL}/api/v1/risk-metrics')
    return response.json()

risk_metrics = get_risk_metrics()

# Create pie chart of risk levels
risk_data = [
    risk_metrics['high_risk_count'],
    risk_metrics['medium_risk_count'],
    risk_metrics['low_risk_count']
]

plt.figure(figsize=(10, 10))
plt.pie(risk_data, labels=['High Risk', 'Medium Risk', 'Low Risk'], autopct='%1.1f%%', startangle=140)
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
plt.title('Risk Distribution')
plt.show()

In [ ]:
plt.pie(risk_data,
        labels=['High Risk', 'Medium Risk', 'Low Risk'],
        colors=['red', 'orange', 'green'],
        autopct='%1.1f%%')
plt.title('Distribution of Risk Levels')
plt.show()

# Plot risk score distribution
dist = risk_metrics['risk_score_distribution']
percentiles = dist['percentiles']

plt.figure(figsize=(10, 6))
plt.boxplot([dist['min'], percentiles['25'], percentiles['50'],
            percentiles['75'], dist['max']])
plt.title('Risk Score Distribution')
plt.ylabel('Risk Score')
plt.show()

## 4. Analyze Circular Patterns

In [ ]:
def get_circular_patterns(min_amount=10000):
    response = requests.get(
        f'{ANALYSIS_URL}/api/v1/analysis/patterns',
        params={'min_amount': min_amount}
    )
    return response.json()

patterns = get_circular_patterns()

# Create network visualization of circular patterns
def visualize_pattern(pattern):
    G = nx.DiGraph()
    
    # Add edges from transactions
    for i in range(len(pattern['accounts'])):
        source = pattern['accounts'][i]
        target = pattern['accounts'][(i + 1) % len(pattern['accounts'])]
        amount = pattern['transactions'][i]['amount']
        G.add_edge(source, target, amount=amount)
    
    # Create visualization
    plt.figure(figsize=(10, 10))
    pos = nx.spring_layout(G)
    
    # Draw nodes
    nx.draw_networkx_nodes(G, pos, node_color='lightblue',
                          node_size=1000)
    
    # Draw edges
    nx.draw_networkx_edges(G, pos, edge_color='gray',
                          width=2, arrowsize=20)
    
    # Add labels
    nx.draw_networkx_labels(G, pos)
    edge_labels = nx.get_edge_attributes(G, 'amount')
    nx.draw_networkx_edge_labels(G, pos,
                                edge_labels={k: f'${v:,.2f}'
                                            for k, v in edge_labels.items()})
    
    plt.title(f'Circular Pattern (Length: {len(pattern['accounts'])})')
    plt.axis('off')
    plt.show()

# Visualize first 3 patterns
for pattern in patterns[:3]:
    visualize_pattern(pattern)

## 5. Entity Risk Analysis

In [ ]:
def analyze_entity_network(entity_id):
    # Get entity's transactions
    response = requests.get(
        f'{BASE_URL}/api/v1/transactions/search',
        params={'entity_id': entity_id}
    )
    transactions = response.json()
    
    # Get risk score
    risk_response = requests.get(f'{SCORING_URL}/api/v1/risk-score/{entity_id}')
    risk_data = risk_response.json()
    
    # Create network visualization
    G = nx.DiGraph()
    
    # Add transactions to graph
    for tx in transactions:
        G.add_edge(tx['source_id'], tx['target_id'],
                  amount=tx['amount'])
    
    # Visualize network
    plt.figure(figsize=(12, 8))
    pos = nx.spring_layout(G)
    
    # Color the target entity differently
    node_colors = ['red' if node == entity_id else 'lightblue'
                  for node in G.nodes()]
    
    nx.draw(G, pos, with_labels=True,
            node_color=node_colors,
            node_size=1000,
            arrowsize=20)
    
    plt.title(f'Transaction Network for {entity_id}\n'
              f'Risk Score: {risk_data["risk_score"]:.2f} '
              f'({risk_data["risk_level"]})')
    plt.show()
    
    # Print summary statistics
    print(f'Summary for {entity_id}:')
    print(f'Total Transactions: {len(transactions)}')
    print(f'Connected Entities: {len(G.nodes()) - 1}')
    print(f'Risk Score: {risk_data["risk_score"]:.2f}')
    print(f'Risk Level: {risk_data["risk_level"]}')

# Analyze a specific entity
analyze_entity_network('ACC_0001')

## 6. Time Series Analysis

In [ ]:
def analyze_temporal_patterns():
    # Get transactions for the last 7 days
    end_date = datetime.now().isoformat()
    start_date = (datetime.now() - timedelta(days=7)).isoformat()
    
    df = get_transactions(start_date, end_date)
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    
    # Create various time series plots
    fig, axes = plt.subplots(3, 1, figsize=(15, 15))
    
    # Daily transaction volume
    daily_volume = df.resample('D', on='timestamp')['amount'].sum()
    daily_volume.plot(ax=axes[0], title='Daily Transaction Volume')
    axes[0].set_ylabel('Total Amount')
    
    # Hourly transaction count
    hourly_count = df.resample('H', on='timestamp').size()
    hourly_count.plot(ax=axes[1], title='Hourly Transaction Count')
    axes[1].set_ylabel('Number of Transactions')
    
    # Average transaction amount by hour of day
    df['hour'] = df['timestamp'].dt.hour
    hourly_avg = df.groupby('hour')['amount'].mean()
    hourly_avg.plot(kind='bar', ax=axes[2],
                    title='Average Transaction Amount by Hour')
    axes[2].set_xlabel('Hour of Day')
    axes[2].set_ylabel('Average Amount')
    
    plt.tight_layout()
    plt.show()

analyze_temporal_patterns()

## 7. Suspicious Pattern Detection

In [ ]:
def detect_suspicious_patterns():
    # Get all high-risk entities
    metrics = get_risk_metrics()
    
    # Get circular patterns
    patterns = get_circular_patterns(min_amount=50000)
    
    print('Summary of Suspicious Activity:')
    print(f'High Risk Entities: {metrics["high_risk_count"]}')
    print(f'Circular Patterns Detected: {len(patterns)}\n')
    
    print('Top Circular Patterns:')
    for i, pattern in enumerate(patterns[:5], 1):
        print(f'\nPattern {i}:')
        print(f'Length: {len(pattern["accounts"])} entities')
        print(f'Total Amount: ${sum(tx["amount"] for tx in pattern["transactions"]):,.2f}')
        print('Entities Involved:', ', '.join(pattern['accounts']))

detect_suspicious_patterns()